In [1]:
%pip install flask requests flask_oauthlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
creds = json.loads(open(os.path.expanduser('~/.credentials/dexcom_pfun-app_glucose.json')).read())
os.environ['DEXCOM_CLIENT_ID'] = creds['client_id']
os.environ['DEXCOM_CLIENT_SECRET'] = creds['client_secret']

In [4]:
import os
from flask import Flask, redirect, url_for, session
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'random_secret_key'  # Change this to a random secret key
oauth = OAuth(app)

dexcom = oauth.remote_app(
    'dexcom',
    consumer_key=os.getenv('DEXCOM_CLIENT_ID'),
    consumer_secret=os.getenv('DEXCOM_CLIENT_SECRET'),
    request_token_params={
        'redirect_uri': 'http://localhost:5000/login/authorized'
    },
    base_url='https://api.dexcom.com/v2/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://api.dexcom.com/v2/oauth2/token',
    authorize_url='https://api.dexcom.com/v2/oauth2/login'
)

@app.route('/')
def index():
    return 'Welcome to Dexcom Wrapper! <a href="/login">Login with Dexcom</a>'

@app.route('/login')
def login():
    return dexcom.authorize(callback=url_for('authorized', _external=True))

@app.route('/login/authorized')
def authorized():
    response = dexcom.authorized_response()
    if response is None or response.get('access_token') is None:
        return 'Access denied: reason={} error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )

    session['dexcom_token'] = (response['access_token'], '')
    me = dexcom.get('users/self')
    return 'Logged in as: {}'.format(me.data)

@dexcom.tokengetter
def get_dexcom_oauth_token():
    return session.get('dexcom_token')

if __name__ == '__main__':
    app.run(debug=True, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/home/robertc/Git/pfun-cma-model/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/robertc/Git/pfun-cma-model/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1052, in launch_instance
    app.initialize(argv)
  File "/home/robertc/Git/pfun-cma-model/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 117, in inner
    return method(app, *args, **kwargs)
  File "/home/robertc/Git/pfun-cma-model/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 689, in initialize
    self.init_sockets()
  File "/home/robertc/Git/pfun-cma-model/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 328, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/home/robertc/Git/pfun-cma-model/.venv/

SystemExit: 1

/home/robertc/Git/pfun-cma-model/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
